# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 2.0 - Panel Integration  
**特性**: 可导出静态 HTML（控件 + 图表都可交互）

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成基本 AI Context** - 数据结构信息
4. **生成完整 AI Prompt** - 包含所有文档（复制给 AI）
5. **选择维度** - 选择分析维度
6. **创建 Panel 仪表盘** - AI 生成分析逻辑
7. **导出 HTML** - 分享给他人

---


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
# ========================================
# 移除 JupyterLab 宽度限制（关键！）
# ========================================
from IPython.display import HTML, display

display(HTML("""
<style>
    /* 核心：解除 JupyterLab 4 的宽度限制 */
    .jp-Notebook { 
        --jp-notebook-max-width: 100% !important; 
    }
    
    /* 确保所有输出容器占满宽度 */
    .jp-Notebook-cell, 
    .jp-Cell-outputWrapper, 
    .jp-OutputArea-output, 
    .jp-OutputArea-child { 
        max-width: none !important; 
        width: 100% !important; 
    }
    
    /* Panel 根容器强制铺满 */
    .bk-root, .bk-root > .bk { 
        width: 100% !important; 
        max-width: none !important; 
    }
</style>
"""))

print("✅ JupyterLab 宽度限制已移除")

# ========================================
# 加载框架
# ========================================
import polars as pl
import plotly.express as px
import panel as pn

from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 2.0 - Panel Integration")


✅ JupyterLab 宽度限制已移除


✅ 环境初始化完成
📚 框架版本: Phase 2.0 - Panel Integration


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行 × {df_df.width} 列")
print(f"📊 数据预览:\n")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行 × 69 列
📊 数据预览:



|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 🤖 Step 3: 生成基本 AI Context

生成数据概览

In [3]:
# 生成基本 AI Context
print("📋 数据结构信息:\n")
print("=" * 80)
print(session.get_ai_context())
print("=" * 80)

print("\n💡 提示: 下一步会生成完整的 AI Prompt")


📋 数据结构信息:

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float64)
- `溢额未决` (Float64

## 📋 Step 4: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

这个 cell 会输出：
- 数据结构信息
- 完整的 Panel Dashboard 使用指南
- 代码模板
- 常见错误对比
- 检查清单


In [4]:
# ========================================
# 📋 给 AI 的完整信息（复制全部输出给 AI）
# ========================================

# 直接执行最新的 Step 4 模板
import os

# 自动找到项目根目录
def find_project_root():
    """向上查找包含 src/ 和 notebooks/templates/ 的项目根目录"""
    current = os.path.abspath('.')
    while current != '/':
        # 检查是否存在项目标志
        if (os.path.exists(os.path.join(current, 'src')) and 
            os.path.exists(os.path.join(current, 'notebooks', 'templates'))):
            return current
        # 向上一级
        current = os.path.dirname(current)
    # 找不到就返回当前目录
    return os.path.abspath('.')

project_root = find_project_root()
step4_path = os.path.join(project_root, 'notebooks', 'templates', 'step4_standalone.py')

# 调试信息（可选，帮助诊断）
# print(f"项目根目录: {project_root}")
# print(f"Step 4 路径: {step4_path}")
# print(f"文件存在: {os.path.exists(step4_path)}")

if os.path.exists(step4_path):
    exec(open(step4_path).read())
else:
    # 如果文件找不到，使用嵌入版本
    print("=" * 80)
    print("📋 **复制以下所有内容给 AI**")
    print("=" * 80)
    print()
    print("## 📊 数据结构")
    print()
    print(session.get_ai_context())
    print()
    print("=" * 80)
    print("## 📚 Panel Dashboard 完整使用指南")
    print("=" * 80)
    print()
    print("⚠️ 规则 0: 禁止硬编码任何维度！")
    print("必须使用: group_col = values.get('_aggregation_dimension')")
    print("必须跳过: if dim == '_aggregation_dimension': continue")
    print()
    print("详细说明见: notebooks/templates/step4_standalone.py")
    print("=" * 80)


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 5: 选择分析维度

选择你想要分析的维度字段

In [5]:
# 👤 选择你要分析的维度
# 修改下面的列表，添加或删除维度

selected_dimensions = [
    '业务年度',
    '业务险种',
    '归属机构' # 取消注释以启用
    # '境内境外',  # 取消注释以启用
]

print(f"✅ 已选择 {len(selected_dimensions)} 个维度:")
for dim in selected_dimensions:
    if dim in df_df.columns:
        n_unique = df_df[dim].n_unique()
        print(f"  - {dim} ({n_unique:,} 个唯一值)")
    else:
        print(f"  ⚠️  {dim} - 不存在于数据中")

print("\n💡 下一步: 创建 Panel 仪表盘")


✅ 已选择 3 个维度:
  - 业务年度 (25 个唯一值)
  - 业务险种 (26 个唯一值)
  - 归属机构 (30 个唯一值)

💡 下一步: 创建 Panel 仪表盘


## 📊 Step 6: 创建 Panel 仪表盘

自动创建仪表盘控件，然后让 AI 生成分析逻辑

In [6]:
# 创建 Panel 仪表盘（自动生成控件）
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘"
)

print("\n💡 下一步: 让 AI 生成分析逻辑")
print("\n📝 给 AI 的提示:")
print("="  * 80)
print("请使用 Panel Dashboard 生成分析代码。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]")
print("="  * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种, 归属机构



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)
  ✅ 归属机构: MultiChoice (30 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (3 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (4 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



💡 下一步: 让 AI 生成分析逻辑

📝 给 AI 的提示:
请使用 Panel Dashboard 生成分析代码。
已选择维度: ['业务年度', '业务险种', '归属机构']
需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]


## 🤖 Step 7: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

AI 应该生成：
1. `@pn.depends` 装饰的更新函数
2. 数据过滤和聚合逻辑
3. Plotly 图表
4. `dashboard.show()` 和 `dashboard.save()`


In [7]:
import polars as pl
import plotly.graph_objects as go
import panel as pn
from datetime import datetime

@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    """
    保费构成堆叠柱状图分析
    严格遵循Panel Dashboard框架规范
    """
    # 1. 获取控件值
    values = {name: widget.value for name, widget in dashboard.widgets.items()}
    
    # 2. 获取聚合维度（必须从控件获取）
    group_col = values.get('_aggregation_dimension', '业务险种')
    
    # 3. 构建过滤条件（必须跳过聚合维度控件）
    filters = []
    for dim, val in values.items():
        if dim == '_aggregation_dimension':
            continue
        
        if isinstance(val, list):
            if '全选' not in val:
                filters.append(pl.col(dim).is_in(val))
        else:
            if val != '全选':
                filters.append(pl.col(dim) == val)
    
    # 4. 应用过滤条件
    filtered = df_df
    for f in filters:
        filtered = filtered.filter(f)
    
    # 5. 确保只分析近5年数据
    current_year = datetime.now().year
    filtered = filtered.filter(
        (pl.col('业务年度').cast(pl.Int32) >= current_year - 5) &
        (pl.col('业务年度').cast(pl.Int32) <= current_year)
    )
    
    # 6. 计算保费构成指标
    result = filtered.group_by(
        pl.col('业务年度').alias('analysis_year'),
        pl.col(group_col).alias('analysis_dim')
    ).agg([
        # 净自留保费（根据用户要求修正）
        (pl.col('自留保费') + pl.col('附加自留保费')).sum().alias('净自留保费'),
        
        # 各项分出保费（堆叠部分）
        pl.col('协议分保费').sum().alias('协议分出'),
        pl.col('成数分保费').sum().alias('成数分出'),
        pl.col('溢额分保费').sum().alias('溢额分出'),
        pl.col('临分分保费').sum().alias('临分分出'),
        
        # 验证用总保费
        pl.col('总保费').sum().alias('总保费')
    ]).sort(['analysis_year', 'analysis_dim'])
    
    # 7. 创建堆叠柱状图
    fig = go.Figure()
    
    # 添加净自留保费（底部）
    fig.add_trace(go.Bar(
        x=result['analysis_year'],
        y=result['净自留保费'],
        name='净自留保费',
        marker_color='#2ca02c',
        hovertemplate='年度: %{x}<br>净自留保费: %{y:,.0f}元<extra></extra>'
    ))
    
    # 添加各项分出保费（按指定顺序堆叠）
    colors = ['#1f77b4', '#ff7f0e', '#d62728', '#9467bd']
    for i, col in enumerate(['协议分出', '成数分出', '溢额分出', '临分分出']):
        fig.add_trace(go.Bar(
            x=result['analysis_year'],
            y=result[col],
            name=col,
            marker_color=colors[i],
            hovertemplate=f'年度: %{{x}}<br>{col}: %{{y:,.0f}}元<extra></extra>'
        ))
    
    # 设置图表布局
    fig.update_layout(
        title=f'近五年保费构成分析（按{group_col}）',
        barmode='stack',
        xaxis_title='业务年度',
        yaxis_title='保费金额（元）',
        height=600,
        hovermode='x unified',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )
    
    return fig

# 绑定更新函数
dashboard.set_update_function(update_dashboard)

# 显示仪表盘
dashboard.show()


Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')